In [19]:
import pandas as pd
import numpy as np
import datetime
import time
import os

### Recortes temporales en Audio
---

In [20]:
audio_path = "../Data Procesada/data_audio.csv"
audio = pd.read_csv(audio_path)
audio.head(3)

,Unnamed: 0,folder,path_audio,file,date,text,start,end
0,0,C5N,../Data/Audio/C5N/C5N2020-08-11-17-59-23.wav,C5N2020-08-11-17-59-23.wav,2020-08-11 17:59:23,el doctor adolfo ruiz stein que había dicho qu...,2020-08-11 17:59:23,2020-08-11 18:00:07
1,1,C5N,../Data/Audio/C5N/C5N2020-08-11-17-59-23.wav,C5N2020-08-11-17-59-23.wav,2020-08-11 17:59:23,muy mal o dejaron sin vacunas a la argentina s...,2020-08-11 18:00:08,2020-08-11 18:00:53
2,2,C5N,../Data/Audio/C5N/C5N2020-08-11-17-59-23.wav,C5N2020-08-11-17-59-23.wav,2020-08-11 17:59:23,demostrar como demostró siempre le importan tr...,2020-08-11 18:00:53,2020-08-11 18:01:38


In [21]:
audio.shape

(528, 8)

In [22]:
audio['start'] = audio['start'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
audio['end'] = audio['end'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))


audio.dtypes

Unnamed: 0             int64
folder                object
path_audio            object
file                  object
date                  object
text                  object
start         datetime64[ns]
end           datetime64[ns]
dtype: object

In [23]:
#Seteo del intervalo delta
delta = 30
td = datetime.timedelta(minutes=delta)


In [39]:
# Join de textos en el intervalo Delta
registrosAudio =  pd.DataFrame(columns=['folder', 'file','deltaStart', 'deltaEnd','textDelta'])

for file in audio['file'].unique():
    
    maskFile =  audio['file'] == file
    
    folder = audio.loc[maskFile, 'folder'].unique()
    
    deltaStart = audio.loc[maskFile, 'start'].min()
    deltaEnd = deltaStart + td
    
    endFile = audio.loc[maskFile, 'end'].max()
    
    while deltaStart <= endFile:
        
        
        maskDelta = (audio['start'] >= deltaStart) & (audio['end'] < deltaEnd)
        
        audio.loc[maskDelta & maskFile,'deltaStart'] = deltaStart
        
        audio.loc[maskDelta & maskFile,'deltaEnd'] = deltaEnd

        text = audio.loc[maskDelta & maskFile,'text'].unique()
        textDelta = ','.join(text)
        newInput=pd.DataFrame([textDelta], columns=['textDelta'])
        newInput['file'] = file
        newInput['folder'] = folder
        newInput['deltaStart'] = deltaStart
        newInput['deltaEnd'] = deltaEnd
        newInput.reset_index(inplace=True)
        
        registrosAudio = registrosAudio.append(newInput)

        deltaStart = deltaStart + td
        deltaEnd = deltaStart + td
        
#registrosAudio.drop(['index'],axis=1)
registrosAudio['file'] = registrosAudio['file'].apply(lambda x: x.split('.')[0]) 

registrosAudio



,folder,file,deltaStart,deltaEnd,textDelta,index
0,C5N,C5N2020-08-11-17-59-23,2020-08-11 17:59:23,2020-08-11 18:29:23,el doctor adolfo ruiz stein que había dicho qu...,0.0
0,C5N,C5N2020-08-11-17-59-23,2020-08-11 18:29:23,2020-08-11 18:59:23,estamos gran equipo esta señora aéreo año bien...,0.0
0,C5N,C5N2020-08-11-17-59-23,2020-08-11 18:59:23,2020-08-11 19:29:23,estaba en el lugar que esta mujer dice que la ...,0.0
0,C5N,C5N2020-08-11-17-59-23,2020-08-11 19:29:23,2020-08-11 19:59:23,la argentina saco enormes ventajas por eso en ...,0.0
0,C5N,C5N2020-08-11-17-59-23,2020-08-11 19:59:23,2020-08-11 20:29:23,como relacionadas con ánimo contento terminand...,0.0
0,C5N,C5N2020-08-14-17-55-49,2020-08-14 17:55:59,2020-08-14 18:25:59,a hola hola muy buenas tardes bienvenidos al d...,0.0
0,C5N,C5N2020-08-14-17-55-49,2020-08-14 18:25:59,2020-08-14 18:55:59,el anterior gobierno no es cierto que sufrió n...,0.0
0,C5N,C5N2020-08-14-17-55-49,2020-08-14 18:55:59,2020-08-14 19:25:59,cama como creen que dicen coronavirus color az...,0.0
0,C5N,C5N2020-08-14-17-55-49,2020-08-14 19:25:59,2020-08-14 19:55:59,argentina la andanada la pole que se le puede ...,0.0
0,C5N,C5N2020-08-14-17-55-49,2020-08-14 19:55:59,2020-08-14 20:25:59,y lo comparo con los periodistas que fuera de ...,0.0


'C5N2020-08-11-17-59-23'

In [24]:
registrosAudio.shape

(17, 5)

### Recortes temporales en video
---

In [25]:
#Concatenado de las tablas de video

In [49]:
videoTablePath = "../Data/Video/TablasVideo/"
data_video = pd.DataFrame()

for videoTable in os.listdir(videoTablePath):
    registro = pd.read_csv(videoTablePath + videoTable)
    #print(registro.shape)
    data_video = pd.concat([data_video, registro])

data_video.drop(['Unnamed: 0'], axis=1, inplace=True)
data_video['file'] = data_video['file'].apply(lambda x: x if x is np.NaN else x.split('.')[0])

In [50]:
data_video.sample(3)

,personaje,fecha_hora,genero,file,fuente
1011,NO IDENTIFICADO,2020-08-18 19:14:47,F,TN2020-08-18-19-00-19,TN
18139,SIN PERSONAS IDENTIFICABLES,2020-08-15 20:57:28,NaN,NaN,C5N
467,franco_mercuriali,2020-08-18 19:06:40,M,TN2020-08-18-19-00-19,TN


In [51]:
##Export dataset
data_video.to_csv("../Data procesada/data_video.csv")

In [ ]:
#Recortes temporales

In [57]:
video_path = "../Data Procesada/data_video.csv"
video = pd.read_csv(video_path)
video.head(3)

,Unnamed: 0,personaje,fecha_hora,genero,file,fuente
0,0,SIN PERSONAS IDENTIFICABLES,2020-08-18 19:00:19,NaN,TN2020-08-18-19-00-19,TN
1,1,SIN PERSONAS IDENTIFICABLES,2020-08-18 19:00:20,NaN,TN2020-08-18-19-00-19,TN
2,2,SIN PERSONAS IDENTIFICABLES,2020-08-18 19:00:21,NaN,TN2020-08-18-19-00-19,TN


In [58]:
video.columns

Index(['Unnamed: 0', 'personaje', 'fecha_hora', 'genero', 'file', 'fuente'], dtype='object')

In [59]:

video['datetime'] = video['fecha_hora'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
video.drop(['Unnamed: 0', 'fecha_hora'], axis=1, inplace=True)
video.dtypes


personaje            object
genero               object
file                 object
fuente               object
datetime     datetime64[ns]
dtype: object

In [60]:
video.head(2)

,personaje,genero,file,fuente,datetime
0,SIN PERSONAS IDENTIFICABLES,NaN,TN2020-08-18-19-00-19,TN,2020-08-18 19:00:19
1,SIN PERSONAS IDENTIFICABLES,NaN,TN2020-08-18-19-00-19,TN,2020-08-18 19:00:20


In [63]:
video.groupby('file')['personaje'].value_counts()

file                   personaje                  
TN2020-08-18-19-00-19  NO IDENTIFICADO                3727
                       SIN PERSONAS IDENTIFICABLES    2470
                       nicolas_winazki                1178
                       carolina_amoroso                853
                       franco_mercuriali               825
                       rolando_barbano                 302
                       maru_duffard                    196
                       sergio_berensztein              188
                       diego_leuco                     126
                       lucia_salinas                   112
                       josep_maria_bartomeu             57
                       alberto_fernandez                32
                       graciela_ciccia                  19
                       mauricio_macri                   17
                       abigail_lassalle                 12
                       luciana_geuna                    11
     

In [64]:
delta = 30
td = datetime.timedelta(minutes=delta)

registros =  pd.DataFrame()

for file in video['file'].unique():
    
    maskFile =  video['file'] == file
    
    deltaStart = video.loc[maskFile, 'datetime'].min()
    deltaEnd = deltaStart + td
    
    endFile = video.loc[maskFile, 'datetime'].max()
    
    while deltaStart <= endFile:
        
        
        maskDelta = (video['datetime'] >= deltaStart) & (video['datetime'] < deltaEnd)
        
        video.loc[maskDelta & maskFile,'deltaStart'] = deltaStart
        
        video.loc[maskDelta & maskFile,'deltaEnd'] = deltaEnd

        newInput = video.loc[maskDelta & maskFile, ['deltaStart', 'deltaEnd', 'file','fuente','personaje']].value_counts()
        
        newInput = pd.DataFrame(newInput)
        newInput.reset_index(inplace=True)
        
        registros = registros.append(newInput)

        deltaStart = deltaStart + td
        deltaEnd = deltaStart + td
        
registros


,deltaStart,deltaEnd,file,fuente,personaje,0
0,2020-08-18 19:00:19,2020-08-18 19:30:19,TN2020-08-18-19-00-19,TN,NO IDENTIFICADO,972
1,2020-08-18 19:00:19,2020-08-18 19:30:19,TN2020-08-18-19-00-19,TN,SIN PERSONAS IDENTIFICABLES,697
2,2020-08-18 19:00:19,2020-08-18 19:30:19,TN2020-08-18-19-00-19,TN,rolando_barbano,197
3,2020-08-18 19:00:19,2020-08-18 19:30:19,TN2020-08-18-19-00-19,TN,franco_mercuriali,99
4,2020-08-18 19:00:19,2020-08-18 19:30:19,TN2020-08-18-19-00-19,TN,carolina_amoroso,51
...,...,...,...,...,...,...
21,2020-08-18 20:30:19,2020-08-18 21:00:19,TN2020-08-18-19-00-19,TN,gerard_pique,1
22,2020-08-18 20:30:19,2020-08-18 21:00:19,TN2020-08-18-19-00-19,TN,horacio_rodriguez_larreta,1
23,2020-08-18 20:30:19,2020-08-18 21:00:19,TN2020-08-18-19-00-19,TN,marcela_pagano,1
24,2020-08-18 20:30:19,2020-08-18 21:00:19,TN2020-08-18-19-00-19,TN,mario_pergolini,1


In [60]:
video['datetime'].min()

Timestamp('2020-08-11 19:01:34')

In [65]:
registrosT = registros.pivot(index=['deltaStart', 'deltaEnd', 'fuente', 'file'], columns=['personaje'])
registrosT.reset_index(inplace=True)
registrosT

deltaStart            deltaEnd fuente  \
personaje                                                  
0         2020-08-18 19:00:19 2020-08-18 19:30:19     TN   
1         2020-08-18 19:30:19 2020-08-18 20:00:19     TN   
2         2020-08-18 20:00:19 2020-08-18 20:30:19     TN   
3         2020-08-18 20:30:19 2020-08-18 21:00:19     TN   

                            file             0                  \
personaje                         lionel_messi NO IDENTIFICADO   
0          TN2020-08-18-19-00-19           1.0           972.0   
1          TN2020-08-18-19-00-19           1.0          1223.0   
2          TN2020-08-18-19-00-19           1.0           698.0   
3          TN2020-08-18-19-00-19           2.0           834.0   

                                                                          \
personaje SIN PERSONAS IDENTIFICABLES abigail_lassalle alberto_fernandez   
0                               697.0              NaN               NaN   
1                               260.0              NaN              30.0   
2                               546.0             12.0               NaN   
3                               967.0              NaN               2.0   

                           ...                                             \
personaje alejandra_malem  ... mario_pergolini maru_duffard matias_posada   
0                     NaN  ...             2.0          5.0           NaN   
1                     NaN  ...             NaN        184.0           NaN   
2                     NaN  ...             NaN          1.0           NaN   
3                     1.0  ...             1.0          6.0           5.0   

                                                                            \
personaje mauricio_macri nicolas_winazki nicolas_winazki_1 rolando_barbano   
0                    2.0            47.0               NaN           197.0   
1                   13.0            71.0               NaN            86.0   
2                    NaN           596.0               1.0             7.0   
3                    2.0           464.0               2.0            12.0   

                                                              
personaje santiago_fioriti santiago_teran sergio_berensztein  
0                      1.0            1.0                1.0  
1                      NaN            3.0              184.0  
2                      1.0            NaN                NaN  
3                      4.0            NaN                3.0  

[4 rows x 34 columns]